#### The highest score achieved so far is 88%
#### below is a figure which will be provided in the report

In [ ]:
import matplotlib.pyplot as plt

data = [0.78, 0.73, 0.73, 0.75, 0.22, 0.73, 0.59, 0.57,0.45]

plt.bar(['Adamic/Adar','Common','Jaccard','Cosine','Preferential','Katz1957','SimRank','Spectral Clustering','Modularity'], data,color=['b', 'r', 'yellow','b','g'])
plt.title('Accuracy on Validation Set for 100 positive link prediction',fontsize=16,color='r')
plt.xlabel('Algorithms Categories')
plt.ylabel('Accuracy')
plt.rcParams['figure.figsize'] = (18.0, 3.0)
plt.show()

# Data Preparation: Train, Test, Validation

In [ ]:
#Import package
import networkx as nx
from networkx.algorithms import tree
import time
import numpy as np
from numpy import mat
import numpy.linalg
from numpy.linalg import eig as eigenValuesAndVectors
import math
import matplotlib.pyplot as plt
import itertools
from scipy import linalg
import sys
import gensim

In [ ]:
# Validation
pos_File='val_positive.txt'
neg_File='val_negative.txt'
Val_G = nx.Graph()
Val_pos_G = nx.Graph()
Val_neg_G = nx.Graph()
with open(pos_File) as file:
    for line in file:
        Node_L, Node_R = [str(x) for x in line.strip().split()]
        Val_G.add_edge(Node_L,Node_R)
        Val_pos_G.add_edge(Node_L,Node_R)
with open(neg_File) as file:
    for line in file:
        Node_L, Node_R = [str(x) for x in line.strip().split()]
        Val_G.add_edge(Node_L,Node_R)
        Val_neg_G.add_edge(Node_L,Node_R)

In [ ]:
# Training

G = nx.Graph() #G = nx.DiGraph()
Node_List = []
FileName = "training.txt"
with open(FileName) as file:
    for line in file:
        Node_L, Node_R = [str(x) for x in line.strip().split()]
        Node_List.append(Node_L)
        Node_List.append(Node_R)
        G.add_edge(Node_L,Node_R)
Node_List = list(set(Node_List))

In [ ]:
#Nodes Dictionary-> G
Nodes_Dictionary={}
NN = list(G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary[NN[n]] = n

## Neighbour_Based

In [ ]:
def Neighbour_Based(v_1,v_2,methods,Graph):
    if methods == 'Adamic/Adar':
        return tuple(nx.adamic_adar_index(Graph,[(v_1,v_2)]))[0][-1]
    v_1_neighbour = list(Graph.neighbors(v_1))
    v_2_neighbour = list(Graph.neighbors(v_2))
    common_neighbour = set(v_1_neighbour) & set(v_2_neighbour) #find common
    if methods == 'Common':
        return len(common_neighbour)
    if methods == 'Jaccard':
        all_neighbour = set(v_1_neighbour) | set(v_2_neighbour) #union all
        return len(common_neighbour) / len(all_neighbour)
    if methods == 'Cosine':
        return len(common_neighbour)/math.sqrt(len(v_1_neighbour)*len(v_2_neighbour))
    if methods == 'Preferential':
        return len(v_1_neighbour)*len(v_2_neighbour)
    return "Wrong Input"

In [ ]:
way = ['Adamic/Adar','Common','Jaccard','Cosine','Preferential']
for w in way:
    Edge_Score = {}
    for edge in Val_G.edges():
        v_1,v_2=edge
        Edge_Score[edge] = Neighbour_Based(v_1,v_2,w,G)
    Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
    count = 0
    for p in Predict:
        a,b = p
        if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
            count=count+1
    print('the accuracy of',w,' is:',count / 100)

## Random Walk-based Methods

### Katz1957

In [ ]:
def Katz1953(v_1,v_2,Graph,max_step=2):
    A = [(math.exp(1)**i) for i in range(max_step,0,-1)]
    beta = [a/sum(A) for a in A]
    
    score = 0
    for i in range(1,max_step+1):
        for path in nx.all_simple_paths(Graph, source=v_1, target=v_2, cutoff=i):
            score = score + beta[i-1]
    return score

In [ ]:
Edge_Score = {}
for edge in Val_G.edges():
    v_1,v_2=edge
    Edge_Score[edge] = Katz1953(v_1,v_2,G)
Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of Katz1953 is:',count / 100)

### SimRank

In [ ]:
def product(list_1,list_2):
    list_3 = []
    for l1 in list_1:
        for l2 in list_2:
            list_3.append((l1,l2))
    return list_3

def simrank(v_1, v_2, _simrank,Graph,C=2,step  = 0, max_iterations=3):
    if (v_1, v_2) not in _simrank:
        _simrank[(v_1, v_2)]=0
        if v_1 == v_2:
            return 1
        if step  == max_iterations:
            return 0
        in_neighbors_v1 = list(Graph.neighbors(v_1))
        in_neighbors_v2 = list(Graph.neighbors(v_2))
        L = len(in_neighbors_v1) * len(in_neighbors_v2)
        if L == 0:
            return 0
        else:
            scale = C / (len(in_neighbors_v1) * len(in_neighbors_v2))
            return scale * sum(simrank(w, x,_simrank, Graph,C,step +1) for w,x in product(in_neighbors_v1,in_neighbors_v2))
    else:
        return 0 

In [ ]:
Edge_Score = {}
#count = 0
for edge in Val_G.edges():
    v_1,v_2=edge
    _simrank = {}
    simrank_value = simrank(v_1,v_2,_simrank,G,C=0.1, max_iterations=2)
    Edge_Score[edge] = simrank_value
    #print(count,':',simrank_value)
    #count = count + 1
Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of SimRank is:',count / 100)

## Clustering
### Spectral Clustering

###### https://networkx.org/documentation/stable/auto_examples/advanced/plot_eigenvalues.html

In [ ]:
Laplacian_Matrix = nx.laplacian_matrix(Val_G) #very sparse

eigenValues, eigenVectors =linalg.eig(Laplacian_Matrix.A)
eigenVectors = eigenVectors.T

idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[idx]

#idx find the index node  in the position of idx
idx_Dictionary = {}
for i in range(len(idx)):
    idx_Dictionary[idx[i]]=i
    
    
#Nodes Dictionary-> Val_G
Nodes_Dictionary_ForClustering={}
NN = list(Val_G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary_ForClustering[NN[n]] = n

In [ ]:
Edge_Score = {}
#count = 0
for edge in Val_G.edges():
    v_1,v_2=edge
    n_1=Nodes_Dictionary_ForClustering[v_1] #Find Index the i-th in G.nodes()
    n_2=Nodes_Dictionary_ForClustering[v_2] #Find Index the i-th in G.nodes()
    idx_1 = idx_Dictionary[n_1] # find the index in the idx
    idx_2 = idx_Dictionary[n_2] # find the index in the idx
    score = numpy.linalg.norm(eigenVectors[:,-50:][idx_1]-eigenVectors[:,-50:][idx_2],1)
    Edge_Score[edge] = -1*score

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of Spectral Clustering is:',count / 100)

### Modularity matrix

In [ ]:
Modularity_Matrix = nx.modularity_matrix(Val_G) #very sparse

eigenValues, eigenVectors =linalg.eig(Modularity_Matrix.A)
eigenVectors = eigenVectors.T

idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[idx]

#idx find the index node  in the position of idx
idx_Dictionary = {}
for i in range(len(idx)):
    idx_Dictionary[idx[i]]=i
    
    
#Nodes Dictionary-> Val_G
Nodes_Dictionary_ForClustering={}
NN = list(Val_G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary_ForClustering[NN[n]] = n

In [ ]:
part = 1100
Edge_Score = {}
#count = 0
for edge in Val_G.edges():
    v_1,v_2=edge
    n_1=Nodes_Dictionary_ForClustering[v_1] #Find Index the i-th in G.nodes()
    n_2=Nodes_Dictionary_ForClustering[v_2] #Find Index the i-th in G.nodes()
    idx_1 = idx_Dictionary[n_1] # find the index in the idx
    idx_2 = idx_Dictionary[n_2] # find the index in the idx
    score = numpy.linalg.norm(eigenVectors[:,:part][idx_1]-eigenVectors[:,:part][idx_2],3)
    Edge_Score[edge] = score

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of Modularity matrix is:',count / 100)

## Random Walk-based Approaches to Node Embedding

### RandomWalk+Ward2Vec = DeepWalk
### address: https://github.com/PingEnLu/Random-Walk
https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
sys.path.append(r'..\Random_Walk')
from random_walks import RandomWalk
random_walk = RandomWalk(G, walk_length=25, num_walks=40, workers=1,p=0.1,q=1)
walklist = random_walk.walks
model = gensim.models.Word2Vec(walklist,window=3,sg=0)

In [ ]:
Edge_Score = {}
#count = 0
for edge in Val_G.edges():
    v_1,v_2=edge
    em_1 = model.wv[int(v_1)]
    em_2 = model.wv[int(v_2)]
    score = em_1.dot(em_2)/math.sqrt(em_1.dot(em_1)+em_2.dot(em_2))
    Edge_Score[edge] = score

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of DeepWalk is:',count / 100)

### Normal Node2Vec
### adress: https://github.com/eliorc/node2vec

In [ ]:
sys.path.append(r'..\node2vec')
from node2vec import Node2Vec

In [ ]:
node2vec = Node2Vec(G, dimensions=100, walk_length=40, num_walks=40, workers=1,p=0.1,q=1)
model = node2vec.fit(window=3,sg=0)

In [ ]:
Edge_Score = {}

for edge in Val_G.edges():
    v_1,v_2=edge
    em_1 = model.wv[v_1]
    em_2 = model.wv[v_2]
    score = em_1.dot(em_2)/math.sqrt(em_1.dot(em_1)+em_2.dot(em_2))
    Edge_Score[edge] = score

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of Node2Vec is:',count / 100)

# Mixed Function
## Spectral Clustering+Cosine Neighbour Score

In [ ]:
Modularity_Matrix = nx.modularity_matrix(Val_G) #very sparse

eigenValues, eigenVectors =linalg.eig(Modularity_Matrix.A)
eigenVectors = eigenVectors.T

idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[idx]

#idx find the index node  in the position of idx
idx_Dictionary = {}
for i in range(len(idx)):
    idx_Dictionary[idx[i]]=i
    
    
#Nodes Dictionary-> Val_G
Nodes_Dictionary_ForClustering={}
NN = list(Val_G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary_ForClustering[NN[n]] = n

In [ ]:
Edge_Score_Cosine = {}
Edge_Score_Clustering = {}
Edge_Score = {}
for edge in Val_G.edges():
    v_1,v_2=edge
    Edge_Score_Cosine[edge] = Neighbour_Based(v_1,v_2,'Cosine',G)
    n_1=Nodes_Dictionary_ForClustering[v_1] #Find Index the i-th in G.nodes()
    n_2=Nodes_Dictionary_ForClustering[v_2] #Find Index the i-th in G.nodes()
    idx_1 = idx_Dictionary[n_1] # find the index in the idx
    idx_2 = idx_Dictionary[n_2] # find the index in the idx
    score = numpy.linalg.norm(eigenVectors[:,-50:][idx_1]-eigenVectors[:,-50:][idx_2],1)
    Edge_Score_Clustering[edge] = -1*score
    Edge_Score[edge] = 0.9*Edge_Score_Clustering[edge] + 1.5*Edge_Score_Cosine[edge]
    
    
Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of (spectral clustering+Cosine Neighbour Score) is:',count / 100)

# Mixed Function
## Spectral Clustering+Node2Vec

In [ ]:
node2vec = Node2Vec(G, dimensions=50, walk_length=20, num_walks=20, workers=1,p=0.1,q=1)
model = node2vec.fit(window=3,sg=1)

In [ ]:
Laplacian_Matrix = nx.laplacian_matrix(Val_G) #very sparse

eigenValues, eigenVectors =linalg.eig(Laplacian_Matrix.A)
eigenVectors = eigenVectors.T

idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[idx]

#idx find the index node  in the position of idx
idx_Dictionary = {}
for i in range(len(idx)):
    idx_Dictionary[idx[i]]=i
    
    
#Nodes Dictionary-> Val_G
Nodes_Dictionary_ForClustering={}
NN = list(Val_G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary_ForClustering[NN[n]] = n

In [ ]:
Edge_Score = {}
Edge_Score_node2Vec = {}
Edge_Score_Clustering = {}
#count = 0
for edge in Val_G.edges():
    v_1,v_2=edge
    
    
    em_1 = model.wv[v_1]
    em_2 = model.wv[v_2]
    score = em_1.dot(em_2)/math.sqrt(em_1.dot(em_1)+em_2.dot(em_2))
    Edge_Score_node2Vec[edge] = score
    
    
    n_1=Nodes_Dictionary_ForClustering[v_1] #Find Index the i-th in G.nodes()
    n_2=Nodes_Dictionary_ForClustering[v_2] #Find Index the i-th in G.nodes()
    idx_1 = idx_Dictionary[n_1] # find the index in the idx
    idx_2 = idx_Dictionary[n_2] # find the index in the idx
    score = numpy.linalg.norm(eigenVectors[:,-50:][idx_1]-eigenVectors[:,-50:][idx_2],1)
    Edge_Score_Clustering[edge] = -1*score
    
    
    Edge_Score[edge] = 4*Edge_Score_node2Vec[edge] + 10*Edge_Score_Clustering[edge]

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of (Modularity clustering+Node2Vec) is:',count / 100)

# Mixed Function
## Modularity Clustering+Cosine Neighbour Score

In [ ]:
Modularity_Matrix = nx.modularity_matrix(Val_G) #very sparse

eigenValues, eigenVectors =linalg.eig(Modularity_Matrix.A)
eigenVectors = eigenVectors.T

idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[idx]

#idx find the index node  in the position of idx
idx_Dictionary = {}
for i in range(len(idx)):
    idx_Dictionary[idx[i]]=i
    
    
#Nodes Dictionary-> Val_G
Nodes_Dictionary_ForClustering={}
NN = list(Val_G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary_ForClustering[NN[n]] = n

In [ ]:
part = 1100
Edge_Score_Cosine = {}
Edge_Score_ModularityClustering = {}
Edge_Score = {}
#count = 0
for edge in Val_G.edges():
    v_1,v_2=edge
    Edge_Score_Cosine[edge] = Neighbour_Based(v_1,v_2,'Cosine',G)
    #Edge_Score_Cosine[edge] = Katz1953(v_1,v_2,G)
    
    n_1=Nodes_Dictionary_ForClustering[v_1] #Find Index the i-th in G.nodes()
    n_2=Nodes_Dictionary_ForClustering[v_2] #Find Index the i-th in G.nodes()
    idx_1 = idx_Dictionary[n_1] # find the index in the idx
    idx_2 = idx_Dictionary[n_2] # find the index in the idx
    score = numpy.linalg.norm(eigenVectors[:,:part][idx_1]-eigenVectors[:,:part][idx_2],3)
    Edge_Score_ModularityClustering[edge] = score
    Edge_Score[edge] = 20*Edge_Score_ModularityClustering[edge] + 2*Edge_Score_Cosine[edge]

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]
count = 0
for p in Predict:
    a,b = p
    if (a,b) in Val_pos_G.edges() or (b,a) in Val_pos_G.edges():
        count=count+1
print('the accuracy of (Modularity clustering + Cosine Neighbour Score) is:',count / 100)

## Prediction: Spectral Clustering+Cosine Neighbour Score

In [ ]:
# Final Prediction
# Loading Test

Test_G = nx.Graph() #G = nx.DiGraph()
Node_List = []
FileName = "test.txt"
with open(FileName) as file:
    for line in file:
        Node_L, Node_R = [str(x) for x in line.strip().split()]
        Node_List.append(Node_L)
        Node_List.append(Node_R)
        Test_G.add_edge(Node_L,Node_R)
Node_List = list(set(Node_List))

In [ ]:
node2vec = Node2Vec(G, dimensions=50, walk_length=20, num_walks=20, workers=1,p=0.1,q=1)
model = node2vec.fit(window=3,sg=1)

In [ ]:
Laplacian_Matrix = nx.laplacian_matrix(Test_G) #very sparse

eigenValues, eigenVectors =linalg.eig(Laplacian_Matrix.A)
eigenVectors = eigenVectors.T

idx = eigenValues.argsort()[::-1]   
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[idx]

#idx find the index node  in the position of idx
idx_Dictionary = {}
for i in range(len(idx)):
    idx_Dictionary[idx[i]]=i
    
    
#Nodes Dictionary-> Val_G
Nodes_Dictionary_ForClustering={}
NN = list(Test_G.nodes())
for n in range(len(NN)):
    Nodes_Dictionary_ForClustering[NN[n]] = n

In [ ]:
Edge_Score = {}
Edge_Score_node2Vec = {}
Edge_Score_Clustering = {}
#count = 0
for edge in Test_G.edges():
    v_1,v_2=edge
    
    
    em_1 = model.wv[v_1]
    em_2 = model.wv[v_2]
    score = em_1.dot(em_2)/math.sqrt(em_1.dot(em_1)+em_2.dot(em_2))
    Edge_Score_node2Vec[edge] = score
    
    
    n_1=Nodes_Dictionary_ForClustering[v_1] #Find Index the i-th in G.nodes()
    n_2=Nodes_Dictionary_ForClustering[v_2] #Find Index the i-th in G.nodes()
    idx_1 = idx_Dictionary[n_1] # find the index in the idx
    idx_2 = idx_Dictionary[n_2] # find the index in the idx
    score = numpy.linalg.norm(eigenVectors[:,-50:][idx_1]-eigenVectors[:,-50:][idx_2],1)
    Edge_Score_Clustering[edge] = -1*score
    
    
    Edge_Score[edge] = 4*Edge_Score_node2Vec[edge] + 10*Edge_Score_Clustering[edge]

Predict = sorted(Edge_Score, key=lambda x: Edge_Score[x],reverse=True)[:100]

In [ ]:
Predict

In [ ]:
result = [pair[0]+' '+pair[1]+'\n' for pair in Predict]
with open('results.txt','w') as f:
    f.writelines(result)